# HW4. Offline Policy Evaluation: Clipped IPS


Сначала повторим шаги подготовки данных и обучения модели с первого дз

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import pickle

Подготовка данных

In [2]:
data = pd.read_csv('../data/data.csv')
# сразу удаляем ненужные колонки - те, которые не будем использовать ни в обучении, ни в оценке политики
data = data.drop(["oaid_hash", "rate0", "rate1"], axis=1)
# плюс фильтруем те данные, у которых banner_id0 не совпадает с banner_id, как написано в задании
data = data[data['banner_id0'] == data['banner_id']]
data.head()

,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,banner_id0,g0,coeff_sum0,banner_id1,g1,coeff_sum1,impressions,clicks
1,2021-09-26 22:54:49.000000,1,1,0,0,1,1,0.054298,-2.657477,269,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,3,0,0,2,0.014096,-3.824875,21,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,0,1,1,3,0.015232,-3.461357,99,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,0,1,0,4,0.051265,-4.009026,11464230,0.032005,-2.828797,1,1
5,2021-09-27 00:06:50.000000,5,5,0,2,2,5,0.337634,-3.222757,37,0.338195,-3.221755,1,1


In [3]:
# так как до этого подготовка данных и обучение модели не предполагали наличие этих новых колонок
# создадим отдельный датасет только с ними, затем будем использовать его для подсчетов ips
test_for_ips = data[pd.to_datetime(data['date_time']) >= pd.to_datetime('2021-10-02')][["banner_id0", "banner_id1", "g0", "g1", "coeff_sum0", "coeff_sum1", "clicks"]].copy()
test_for_ips.head()

,banner_id0,banner_id1,g0,g1,coeff_sum0,coeff_sum1,clicks
164,76,401,0.055551,0.030272,-2.926980,-3.390642,1
166,46,11464251,0.017521,0.085038,-1.377320,-3.329596,1
168,76,11464252,0.171074,0.079034,-3.112081,-1.907685,1
169,46,0,0.017439,0.017624,-2.493974,-3.889516,1
359,2,49,0.020414,0.068041,-2.154111,-3.088063,1


In [4]:
# а данные для обучения и теста модели подготовим почти идентично с тем, как готовили к первому дз
# по факту просто убираю колонку campaign_clicks
# плюс сразу готовлю два отдельных тестовых датасета X_test и X_test_counter
# - реальный тест (banner_id = banner_id0) и тест, где banner_id = banner_id1

def feature_engineering(data: pd.DataFrame):
    data = data.copy()
    data = data.drop(["banner_id0", "g0", "g1", "coeff_sum0", "coeff_sum1", "campaign_clicks"], axis=1)

    # создаем колонки час и день недели
    data['date_time'] = pd.to_datetime(data['date_time'])
    data['day_of_week'] = data['date_time'].dt.dayofweek
    data['hour'] = data['date_time'].dt.hour
    data['date'] = data['date_time'].dt.date
    # эти колонки будем использовать для обучения (кроме date и clicks и banner_id1)
    data = data[['day_of_week', 'hour', 'os_id', 'country_id', 'zone_id', 'banner_id', 'date', 'clicks', "banner_id1"]]
    # здесь добавляю единицу, чтобы далее чуть более корректно обрабатывались нечастые случаи (и те, которые не встречались) в OneHotEncoder
    # там они будут заменяться нулями, поэтому сдвигаю оригинальные значения на 1
    data[['day_of_week', 'hour', 'os_id', 'country_id', 'zone_id', 'banner_id']] += 1
    # делим на train и test
    train = data[data['date'] < pd.to_datetime('2021-10-02')]
    test = data[data['date'] == pd.to_datetime('2021-10-02')]
    # лейблы
    y_train = np.array(train['clicks'])
    y_test = np.array(test['clicks'])
    test_counter = test.copy()
    test_counter['banner_id'] = test_counter['banner_id1']
    train = train.drop(['date', "clicks", "banner_id1"], axis=1)
    test = test.drop(['date', "clicks", "banner_id1"], axis=1)
    test_counter = test_counter.drop(['date', "clicks", "banner_id1"], axis=1)
    # закодируем все фичи с помощью one-hot-encoding
    enc = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=True, min_frequency=10)
    X_train = enc.fit_transform(train)
    X_test = enc.transform(test)
    X_test_counter = enc.transform(test_counter)

    return X_train, X_test, X_test_counter, y_train, y_test

X_train, X_test, X_test_counter, y_train, y_test = feature_engineering(data)
print(X_train.shape, X_test.shape, X_test_counter.shape, y_train.shape, y_test.shape)

/tmp/ipykernel_16061/4279608246.py:21: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  train = data[data['date'] < pd.to_datetime('2021-10-02')]
/tmp/ipykernel_16061/4279608246.py:22: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  test = data[data['date'] == pd.to_datetime('2021-10-02')]
/home/alyona-pestova/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [0, 4, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/alyona-pestova/.local/lib/python3.10/site-packages/sklearn/pr

(12056598, 2888) (1890562, 2888) (1890562, 2888) (12056598,) (1890562,)


Модель логистической регрессии. Здесь уже использую параметры, подобранные на пред.домашке, хотя и данные немного отличаются - нет колонки  campaign_clicks, как и написано в задании, однако заново прогонять кросс-валидацию слишком долго.

In [5]:
# обучила модель до этого, далее загружаю ее
# model = LogisticRegression(C=1, solver='liblinear', penalty='l1')
# model.fit(X_train, y_train)
# pickle.dump(model, open('logreg.pkl', 'wb'))

In [6]:
del X_train, y_train, data

Оценим модель на тесте (на последнем дне) стандартными метриками

In [7]:
model = pickle.load(open('logreg.pkl', 'rb'))

In [8]:
from sklearn.metrics import log_loss, roc_auc_score

def eval_prediction(y_pred, y_test):
    loss = log_loss(y_test, y_pred)
    roc_score = roc_auc_score(y_test, y_pred)
    print(f"Log loss: {loss}, roc-auc: {roc_score}")
preds = model.predict_proba(X_test)
eval_prediction(preds[:, 1], y_test)

Log loss: 0.13532973588031427, roc-auc: 0.7906495312129354


## Clipped IPS

Сначала посчитаем \pi_0 для всех наблюдений из теста.


Нужно посчитать вероятность того, что одна случайная величина больше другой, то есть мы имеем:

$$X \sim N(\text{coeff0}, g0^2), Y \sim N(\text{coeff1}, g1^2)$$, а их разность

$$F_{Y-X} = Y - X \sim N(\text{coeff1} - \text{coeff0}, g0^2 + g1^2)$$

нужно посчитать $$P(X > Y) = P(Y - X < 0) = F_{Y-X}(0) = \Phi\left(\dfrac{0 - (\text{coeff1} - \text{coeff0})}{\sqrt{g0^2 + g1^2}}\right) = \Phi\left(\dfrac{(\text{coeff0} - \text{coeff1})}{\sqrt{g0^2 + g1^2}}\right)$$

In [9]:
# заметила, что в колонке g1 есть отрицательные значения, что странно для стандартного отклонения
# видимо, это опечатка - решила взять его по модулю
test_for_ips['g1'] = test_for_ips['g1'].apply(abs)

In [10]:
import scipy
def calc_pi(coeff_sum0, coeff_sum1, g0, g1):
    # считаем вероятность, что N(coeff_sum0 - coeff_sum1, g0**2 + g1**2) > 0
    mu = coeff_sum0 - coeff_sum1
    sd = np.sqrt(g0 ** 2 + g1 ** 2) + 0.000001
    # считаем вероятность того, что значение для нулевого банера больше, чем для первого
    return scipy.stats.norm.cdf(mu / sd)

np.random.seed(10)
test_for_ips['pi_0'] = test_for_ips.apply(lambda x: calc_pi(x.coeff_sum0, x.coeff_sum1, x.g0, x.g1), axis=1)

Теперь нужно посчитать \pi_1, для этого сначала прогоним через обученную модель оригинальные тестовые данные и получим вектор coeff_sum0 как логиты от предсказанных вероятностей модели.
Затем, сделаем то же для наших counterfactual данных - тех, где banner_id=banner_id1, тем же способом получим coeff_sum1. Далее, используя функцию выше посчитаем \pi_1 с полученными coeff_sum0 и coeff_sum1 и исходными g0 и g1.

In [11]:
def logit(p):
    return np.log(p) - np.log(1 - p)

# логиты для оригинальных данных
test_for_ips["preds0"] = logit(model.predict_proba(X_test)[:, 1])
# логиты для counterfactual данных
test_for_ips["preds1"] = logit(model.predict_proba(X_test_counter)[:, 1])

# считаем \pi_1
test_for_ips["pi_1"] = test_for_ips.apply(lambda x: calc_pi(x.preds0, x.preds1, x.g0, x.g1), axis=1)

In [12]:
# плюс оказалось, что для некоторых наблюдений в колонках g0 и coeff_sum1 стоят NA
# для дальнейших рассчетов они нужны, поэтому удалю их
print(test_for_ips[test_for_ips['g1'].isna()])
test_for_ips = test_for_ips[~test_for_ips['g1'].isna()]

          banner_id0  banner_id1        g0  g1  coeff_sum0  coeff_sum1  \
2237              53    11464658  0.053149 NaN   -2.680475         NaN   
2320              53    11464674  0.060256 NaN   -2.683287         NaN   
3632              53    11464937  0.053055 NaN   -2.679521         NaN   
6396              25    11465428  0.085486 NaN   -2.956483         NaN   
10554             53    11466272  0.056577 NaN   -2.673896         NaN   
...              ...         ...       ...  ..         ...         ...   
15814927         130    14622326  0.150797 NaN   -4.626632         NaN   
15816566         141    14622636  0.074683 NaN   -3.462076         NaN   
15817556          28    14622810  0.014501 NaN   -2.951138         NaN   
15818967          28    14623093  0.007713 NaN   -2.782285         NaN   
15821364         141    14623582  0.057890 NaN   -3.317025         NaN   

          clicks  pi_0    preds0    preds1  pi_1  
2237           1   NaN -0.149904 -1.493457   NaN  
2320     

теперь осталось только посчитать отношение pi_0 и pi_1, взять минимум из этого отношения и лямбды (равной 10), перемножить с вектором награды (который у нас совпадает с колонкой clicks) и взять среднее.

In [13]:
ratio = (test_for_ips["pi_0"].to_numpy() + 0.00001) / (test_for_ips["pi_1"].to_numpy() + 0.00001)
ratio = np.clip(ratio, 0, 10)
v = ratio * test_for_ips["clicks"].to_numpy()
V = np.mean(v)
print('cips:', V)

cips: 0.08673718055443559
